Transformer

## Task Description

Learn the internal working mechanisms of Transformer more closely.  

You need to **implement TransformerEncoder and TransformerDecoder with their layers** by your own. Using your TransformerEncoder and TransformerDecoder **build a Transformer model and train it** on the task of translation from English to French.

If you do not know how Encoder and Decoder could be implemented, check the original paper [Attention is All You Need](https://arxiv.org/pdf/1706.03762) that describes the implementation of Encoder and Decoder in details.

To implement the layers you can use the following modules provided by PyTorch:

1. [MultiheadAttention](https://pytorch.org/docs/stable/generated/torch.nn.MultiheadAttention.html)
2. [FFN](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html)
3. [ReLU](https://pytorch.org/docs/stable/generated/torch.nn.ReLU.html)
4. [LayerNorm](https://pytorch.org/docs/stable/generated/torch.nn.LayerNorm.html)
5. [Dropout](https://pytorch.org/docs/stable/generated/torch.nn.Dropout.html)

However, you are **not allowed** to use PyTorch implementation of Transformer, TransformerEncoder, TransformerDecoder and their layers!

You can use the code from the lab to build a Transformer from your layers and to train it on the translation task.

## Grading Criteria

* **TransformerEncoderLayer** is implemented - 1 point
* **TransformerEncoder** is implemented - 1 point
* **TransformerDecoderLayer** is implemented - 1 point
* **TransformerDecoder** is implemented - 1 point
* Transformer **training runs successfully** without any errors - 1 point


## Machine Translation

Vanilla Transformer can be used to implement a translator from one language to another. This task is called Machine Translation (MT). Here we will implement Vanilla Transformer using PyTorch to translate English text to French.

First, let us install necessary packages and import necessary libraries

In [1]:
!pip install datasets
!pip install sacrebleu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.0/104.0 kB 5.7 MB/s eta 0:00:00


In [2]:
import torch
from torch import nn
import torch.optim as optim
import math

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

## Positional Encoding

Now, let's implement calculation of the positional encoding

In [4]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        # Initialize the PositionalEncoding class
        super(PositionalEncoding, self).__init__()

        # Create a tensor to store positional encodings with shape (max_len, d_model)
        # max_len: maximum length of the input sequence
        # d_model: dimension of the model embeddings (matches token embedding size)
        self.encoding = torch.zeros(max_len, d_model)

        # Create a tensor of positions from 0 to max_len-1 with shape (max_len, 1)
        # Each row represents the position of a word in the sequence
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)

        # Compute the denominator for the sine and cosine positional encodings
        # This creates a scaling factor based on the position in the model dimension (d_model)
        # d_model is divided by 2 since we alternate between sine and cosine encodings
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))

        # Apply sine to even dimensions (0, 2, 4, ...) of the positional encoding
        # This is done by multiplying the position by the div_term and applying sin
        self.encoding[:, 0::2] = torch.sin(position * div_term)

        # Apply cosine to odd dimensions (1, 3, 5, ...) of the positional encoding
        # This is done by multiplying the position by the div_term and applying cos
        self.encoding[:, 1::2] = torch.cos(position * div_term)

        # Add an extra dimension at the start to match the batch size during training
        # Shape becomes (1, max_len, d_model), allowing it to be added to input sequences
        self.encoding = self.encoding.unsqueeze(0)

    def forward(self, x):
        # Add the positional encoding to the input tensor (x), ensuring that it is on the same device (CPU or GPU)
        # The encoding tensor is sliced to match the sequence length of the input (x.size(1))
        # This step adds positional information to the input embeddings
        x = x + self.encoding[:, :x.size(1), :].to(x.device)
        return x


## Transformer

TASK: Implement Transformer Encoder Layer

In [5]:
class TransformerEncoderLayer(nn.Module):
    def __init__(self, d_model, nhead, dim_feedforward, dropout=0.1):
        super(TransformerEncoderLayer, self).__init__()

        # Multi-Head Self-Attention mechanism
        self.self_attn = nn.MultiheadAttention(d_model, nhead, dropout=dropout, batch_first=True)

        # Feedforward network
        self.feedforward = nn.Sequential(
            nn.Linear(d_model, dim_feedforward),
            nn.ReLU(),
            nn.Linear(dim_feedforward, d_model)
        )

        # Layer Normalization and Dropout layers
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)

    def forward(self, src, src_mask=None, src_key_padding_mask=None):
        # Self-attention layer with residual connection and layer normalization
        attn_output, _ = self.self_attn(src, src, src, attn_mask=src_mask,
                                        key_padding_mask=src_key_padding_mask)
        src = src + self.dropout1(attn_output)
        src = self.norm1(src)

        # Feed-forward network with residual connection and layer normalization
        ff_output = self.feedforward(src)
        src = src + self.dropout2(ff_output)
        src = self.norm2(src)

        return src


TASK: Implement Transformer Decoder Layer

In [6]:
class TransformerDecoderLayer(nn.Module):
    def __init__(self, d_model, nhead, dim_feedforward, dropout=0.1):
        super(TransformerDecoderLayer, self).__init__()

        # Masked Multi-Head Self-Attention (target attends to itself)
        self.self_attn = nn.MultiheadAttention(d_model, nhead, dropout=dropout, batch_first=True)

        # Multi-Head Attention (target attends to encoder's output, i.e., memory)
        self.multihead_attn = nn.MultiheadAttention(d_model, nhead, dropout=dropout, batch_first=True)

        # Feedforward network
        self.feedforward = nn.Sequential(
            nn.Linear(d_model, dim_feedforward),
            nn.ReLU(),
            nn.Linear(dim_feedforward, d_model)
        )

        # Layer Normalization and Dropout layers
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.norm3 = nn.LayerNorm(d_model)
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)
        self.dropout3 = nn.Dropout(dropout)

    def forward(self, tgt, memory, tgt_mask=None, memory_mask=None,
                tgt_key_padding_mask=None, memory_key_padding_mask=None):

        # Masked self-attention layer (decoder attends to previous positions in tgt)
        tgt2, _ = self.self_attn(tgt, tgt, tgt, attn_mask=tgt_mask,
                                 key_padding_mask=tgt_key_padding_mask)
        tgt = tgt + self.dropout1(tgt2)
        tgt = self.norm1(tgt)

        # Multi-head attention layer (decoder attends to encoder's memory)
        tgt2, _ = self.multihead_attn(tgt, memory, memory, attn_mask=memory_mask,
                                      key_padding_mask=memory_key_padding_mask)
        tgt = tgt + self.dropout2(tgt2)
        tgt = self.norm2(tgt)

        # Feed-forward network
        tgt2 = self.feedforward(tgt)
        tgt = tgt + self.dropout3(tgt2)
        tgt = self.norm3(tgt)

        return tgt


TASK: Implement Transformer Encoder

In [7]:
class TransformerEncoder(nn.Module):
    def __init__(self, encoder_layer, num_layers):
        super(TransformerEncoder, self).__init__()

        # Create a list of encoder layers (each identical)
        self.layers = nn.ModuleList([encoder_layer for _ in range(num_layers)])

        # Optionally, you can add a final layer normalization
        self.norm = nn.LayerNorm(encoder_layer.self_attn.embed_dim)

    def forward(self, src, mask=None, src_key_padding_mask=None):
        # Pass the input through each encoder layer in sequence
        for layer in self.layers:
            src = layer(src, src_mask=mask, src_key_padding_mask=src_key_padding_mask)

        # Optionally normalize the output
        if self.norm is not None:
            src = self.norm(src)

        return src


TASK: Implement Transformer Decoder

In [8]:
class TransformerDecoder(nn.Module):
    def __init__(self, decoder_layer, num_layers):
        super(TransformerDecoder, self).__init__()

        # Stack multiple decoder layers
        self.layers = nn.ModuleList([decoder_layer for _ in range(num_layers)])

        # Optional final layer normalization
        self.norm = nn.LayerNorm(decoder_layer.self_attn.embed_dim)

    def forward(self, tgt, memory, tgt_mask=None, memory_mask=None,
                tgt_key_padding_mask=None, memory_key_padding_mask=None):

        # Pass the target sequence and memory through each decoder layer
        for layer in self.layers:
            tgt = layer(tgt, memory, tgt_mask=tgt_mask, memory_mask=memory_mask,
                        tgt_key_padding_mask=tgt_key_padding_mask,
                        memory_key_padding_mask=memory_key_padding_mask)

        # Apply final layer normalization if defined
        if self.norm is not None:
            tgt = self.norm(tgt)

        return tgt


Here we implement the Transformer architecture in `nn.Module`.

In [9]:
class TransformerModel(nn.Module):
    def __init__(
        self,
        src_vocab_size,        # Vocabulary size of the source language (input)
        tgt_vocab_size,        # Vocabulary size of the target language (output)
        d_model=512,           # Dimension of model embeddings (default 512)
        nhead=8,               # Number of attention heads in multi-head attention (default 8)
        num_encoder_layers=6,  # Number of layers in the Transformer encoder (default 6)
        num_decoder_layers=6,  # Number of layers in the Transformer decoder (default 6)
        dim_feedforward=2048,  # Dimension of the feedforward network inside the Transformer (default 2048)
        dropout=0.1            # Dropout rate (default 0.1)
    ):
        # Initialize the nn.Module parent class
        super(TransformerModel, self).__init__()

        # Source embedding layer that converts input tokens to embeddings of size d_model
        self.src_embedding = nn.Embedding(src_vocab_size, d_model)

        # Target embedding layer that converts target tokens to embeddings of size d_model
        self.tgt_embedding = nn.Embedding(tgt_vocab_size, d_model)

        # Positional encoding for the embeddings to encode the positions of tokens in the sequence
        self.positional_encoding = PositionalEncoding(d_model)

        # Create a single layer of Transformer encoder
        encoder_layer = TransformerEncoderLayer(d_model, nhead, dim_feedforward, dropout)

        # Stack multiple encoder layers (num_encoder_layers defines how many)
        self.transformer_encoder = TransformerEncoder(encoder_layer, num_encoder_layers)

        # Create a single layer of Transformer decoder
        decoder_layer = TransformerDecoderLayer(d_model, nhead, dim_feedforward, dropout)

        # Stack multiple decoder layers (num_decoder_layers defines how many)
        self.transformer_decoder = TransformerDecoder(decoder_layer, num_decoder_layers)

        # Final linear layer to map the decoder output to the target vocabulary size
        # The output dimension is the size of the target vocabulary (tgt_vocab_size)
        self.fc_out = nn.Linear(d_model, tgt_vocab_size)

        # Store the model embedding dimension (d_model) for scaling later
        self.d_model = d_model

    def forward(
        self,
        src,                    # Source sequence (input)
        tgt,                    # Target sequence (input for the decoder)
        src_mask,               # Mask for the source sequence (to avoid attending to padding tokens)
        tgt_mask,               # Mask for the target sequence (prevents attention to future tokens)
        src_padding_mask,       # Padding mask for the source (to avoid attention to padding)
        tgt_padding_mask,       # Padding mask for the target (to avoid attention to padding)
        memory_key_padding_mask # Padding mask for the memory (encoder output) in the decoder
    ):
        # Embed the source sequence and scale by sqrt of d_model for stable gradients
        src_emb = self.src_embedding(src) * math.sqrt(self.d_model)

        # Embed the target sequence and scale by sqrt of d_model
        tgt_emb = self.tgt_embedding(tgt) * math.sqrt(self.d_model)

        # Add positional encodings to the source embeddings
        src_emb = self.positional_encoding(src_emb)

        # Add positional encodings to the target embeddings
        tgt_emb = self.positional_encoding(tgt_emb)

        # Pass the source embeddings through the Transformer encoder
        # The encoder produces a memory representation for the source sequence
        memory = self.transformer_encoder(src_emb, mask=src_mask, src_key_padding_mask=src_padding_mask)

        # Pass the target embeddings and the memory (encoder output) through the Transformer decoder
        # The decoder attends to both the target sequence and the encoder's memory
        output = self.transformer_decoder(
            tgt_emb, memory, tgt_mask=tgt_mask, memory_mask=src_mask,
            tgt_key_padding_mask=tgt_padding_mask,
            memory_key_padding_mask=memory_key_padding_mask
        )


        # Apply the final linear layer to map the decoder output to the target vocabulary
        output = self.fc_out(output)

        # Return the final output (logits over the target vocabulary)
        return output


## Mask Generation Functions

It is also necessary to implement functions that create masks for the input data

In [10]:
def generate_square_subsequent_mask(sz):
    # Create a square matrix of size (sz, sz) filled with ones above and on the diagonal, zeros below the diagonal.
    mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)

    # Convert the mask to a float tensor and fill the zeros with -inf (indicating they should be ignored in attention)
    # and ones with 0.0 (indicating those positions can be attended to).
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))

    # Return the final mask
    return mask


In [11]:
def create_padding_mask(seq, pad_idx):
    # Return a boolean mask where the elements in the sequence that are equal to the pad_idx (padding index) are True,
    # indicating that those positions are padding tokens.
    return (seq == pad_idx).float()


## Machine Translation

After we have defined the model, now we can train the model to translate the text from English to French.

So, let's start with loading the dataset

In [12]:
from datasets import load_dataset

texts = load_dataset("opus_books", "en-fr", split='train')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/28.1k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/127085 [00:00<?, ? examples/s]

Here is just some unwrapping of the data. Nothing that important.

In [13]:
src_lang = 'en'
tgt_lang = 'fr'

texts = texts.map(
    lambda x: {
        'en': x['translation']['en'],
        'fr': x['translation']['fr']
    }
)

Map:   0%|          | 0/127085 [00:00<?, ? examples/s]

Definition of how many data samples should be used for training and testing

In [14]:
# Total number of samples
max_size = len(texts)
# max_size = 10000

# Percentage of samples from the total number that will be used for training
train_frac = 0.8 # Percen

# Calculation of exact number of training and testing samples
train_size = int(train_frac * max_size)
test_size = max_size - train_size

# Selection of the data for training and testing according to the number
# of training and testing samples
train_texts = texts.select(range(train_size))
test_texts = texts.select(range(train_size, train_size+test_size))

Wrap the dataset into PyTorch dataset in order to load data with DataLoader

In [15]:
class TranslationDataset(torch.utils.data.Dataset):
    def __init__(self, data, src_lang='en', tgt_lang='fr'):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return {
            "src": self.data[idx][src_lang],
            "tgt": self.data[idx][tgt_lang]
        }

train_dataset = TranslationDataset(train_texts)
test_dataset = TranslationDataset(test_texts)

For the sake of simplicity, we will use the ready-to-use tokenizer of English and French texts

In [16]:
!pip install sentencepiece


In [17]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-fr")

# Add Begining of Sentence (BOS) token since it is absent in the tokenizer
tokenizer.add_special_tokens({'bos_token': '<BOS>'})

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/778k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/802k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.34M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1

Create training and testing DataLoaders. Implement `collate_fn` in which we tokenize the batch of text using [`tokenizer.batch_encode_plus`](https://huggingface.co/docs/transformers/v4.45.2/en/internal/tokenization_utils#transformers.PreTrainedTokenizerBase.batch_encode_plus)

In [18]:
from torch.utils.data import DataLoader

# Define the maximum sequence length and batch size
max_length = 64
batch_size = 8

# Custom function to process and batch data
def collate_fn(batch):
    # Prepend "<BOS>" token to each source text in the batch and tokenize it
    tokenized_src_texts = tokenizer.batch_encode_plus(
        ["<BOS> " + item['src'] for item in batch],  # Add "<BOS>" to the source text before tokenization
        padding="max_length",                        # Pad sequences to the max_length
        max_length=max_length,                       # Define the maximum length for padding/truncation
        truncation=True,                             # Truncate sequences longer than max_length
        return_tensors="pt"                          # Return a PyTorch tensor
    )['input_ids']                                   # Extract the 'input_ids' (token IDs)

    # Prepend "<BOS>" token to each target text in the batch and tokenize it
    tokenized_tgt_texts = tokenizer.batch_encode_plus(
        ["<BOS> " + item['tgt'] for item in batch],  # Add "<BOS>" to the target text before tokenization
        padding="max_length",                        # Pad sequences to the max_length
        max_length=max_length,                       # Define the maximum length for padding/truncation
        truncation=True,                             # Truncate sequences longer than max_length
        return_tensors="pt"                          # Return a PyTorch tensor
    )['input_ids']                                   # Extract the 'input_ids' (token IDs)

    # Return the tokenized and padded source and target texts as tensors
    return {
        'src': tokenized_src_texts,                  # Tokenized source sequences tensor
        'tgt': tokenized_tgt_texts                   # Tokenized target sequences tensor
    }

# Create DataLoader for training data
train_dataloader = DataLoader(
    train_dataset,                                   # Training dataset
    batch_size=batch_size,                           # Batch size for training
    collate_fn=collate_fn,                           # Custom collate function for tokenizing and batching
    shuffle=True                                     # Shuffle the data each epoch
)

# Create DataLoader for test data
test_dataloader = DataLoader(
    test_dataset,                                    # Test dataset
    batch_size=batch_size,                           # Batch size for testing
    collate_fn=collate_fn,                           # Custom collate function for tokenizing and batching
    shuffle=False                                    # Do not shuffle the test data
)


Create an instance of the model, optimizer, and loss

In [19]:
# Retrieve special token indices from the tokenizer
pad_idx = tokenizer.pad_token_id  # Padding token index
bos_idx = tokenizer.bos_token_id  # Beginning of sequence token index
eos_idx = tokenizer.eos_token_id  # End of sequence token index

# Define vocabulary size, adding 1 due to the new BOS token
vocab_size = tokenizer.vocab_size + 1

# Initialize the Transformer model
model = TransformerModel(
    vocab_size,               # Source vocabulary size (input)
    vocab_size,               # Target vocabulary size (output)
    d_model=512,              # Dimensionality of the model (embedding size)
    nhead=8,                  # Number of attention heads in the multi-head attention mechanism
    num_encoder_layers=6,     # Number of layers in the encoder
    num_decoder_layers=6,     # Number of layers in the decoder
    dim_feedforward=2048,     # Dimension of the feedforward network inside each Transformer layer
    dropout=0.1               # Dropout rate for regularization
).to(device)                  # Move the model to the specified device (e.g., GPU or CPU)

# Initialize the optimizer (Adam optimizer with a learning rate of 0.0001)
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# Define the loss function (Cross-Entropy Loss) with the padding token ignored
criterion = nn.CrossEntropyLoss(ignore_index=pad_idx)


Check how many parameters our model has

In [20]:
sum(p.numel() for p in model.parameters())

98833019

Define the function for training the model

In [21]:
from tqdm import tqdm  # Import tqdm for progress bars

def train_model(
    model,                # The Transformer model
    train_loader,         # DataLoader for the training data
    optimizer,            # Optimizer (Adam in this case)
    criterion,            # Loss function (Cross-Entropy Loss)
    pad_idx,              # Padding token index to ignore in loss calculation
    device,               # Device (CPU or GPU)
    num_epochs=1         # Number of epochs to train the model
):
    model.train()  # Set the model to training mode
    for epoch in range(num_epochs):  # Loop through each epoch
        total_loss = 0  # Initialize total loss for the current epoch

        pbar = tqdm(train_loader)  # Wrap train_loader with tqdm for progress display
        for i, batch in enumerate(pbar):  # Loop through each batch in the training data
            # Move the source and target sequences to the device (e.g., GPU)
            src, tgt = batch['src'].to(device), batch['tgt'].to(device)

            # Prepare input and output sequences for teacher forcing
            tgt_input = tgt[:, :-1]  # Target input (remove the last token)
            tgt_output = tgt[:, 1:]  # Target output (shifted by one token)

            # Generate masks for attention
            src_mask = None  # No source mask in this case
            tgt_mask = generate_square_subsequent_mask(  # Generate mask for target input to prevent attending to future tokens
                tgt_input.size(1)
            ).to(device)

            # Create padding masks for both source and target sequences
            src_padding_mask = create_padding_mask(src, pad_idx).to(device)
            tgt_padding_mask = create_padding_mask(tgt_input, pad_idx).to(device)
            memory_padding_mask = create_padding_mask(src, pad_idx).to(device)

            optimizer.zero_grad()  # Zero gradients before backpropagation

            # Forward pass through the model
            output = model(
                src, tgt_input, src_mask, tgt_mask,  # Source and target with their masks
                src_padding_mask, tgt_padding_mask, memory_padding_mask  # Padding masks for the attention mechanism
            )
            output = output.view(-1, output.size(-1))  # Reshape the model's output for calculating loss
            tgt_output = tgt_output.contiguous().view(-1)  # Flatten the target output for loss calculation

            # Compute the loss
            loss = criterion(output, tgt_output)  # Cross-entropy loss
            loss.backward()  # Backpropagate the gradients
            optimizer.step()  # Update the model's parameters

            total_loss += loss.item()  # Accumulate total loss for this epoch

            # Update the progress bar description with the current average loss
            pbar.set_description(f"Loss: {total_loss / (i + 1)}")


# Call the training function with one epoch of training
train_model(
    model,                 # The Transformer model
    train_dataloader,      # DataLoader for training data
    optimizer,             # Adam optimizer
    criterion,             # Cross-Entropy Loss
    pad_idx,               # Padding index to ignore in the loss
    device,                # Device (GPU or CPU)
    num_epochs=1           # Number of epochs to train
)


Loss: 6.372943062909479: 100%|██████████| 12709/12709 [24:02<00:00,  8.81it/s]


Check how the model translates the English text

In [22]:
def translate_sentence(model, tokenizer, sentence, max_length, pad_idx, device):
    # Tokenize the input sentence
    model.eval()  # Set the model to evaluation mode
    tokens = tokenizer.encode(sentence, return_tensors="pt").to(device)

    # Ensure tokens are not longer than max_length
    tokens = tokens[:, :max_length]

    # Prepare the input tensor (add batch dimension if necessary)
    src = tokens

    # Initialize the decoder input with <BOS> token
    tgt = torch.tensor([[bos_idx]], dtype=torch.long).to(device)

    # Create padding masks
    src_padding_mask = create_padding_mask(src, pad_idx).to(device)

    # Generate a translation by decoding one token at a time
    for _ in range(max_length):

        with torch.no_grad():
            output = model(
                src, tgt, src_mask=None, tgt_mask=None,
                src_padding_mask=src_padding_mask, tgt_padding_mask=None,
                memory_key_padding_mask=src_padding_mask
            )

        # Get the last predicted token (greedy decoding)
        next_token = output[:, -1, :].argmax(dim=-1).unsqueeze(0)

        # Concatenate predicted token to target sequence
        tgt = torch.cat([tgt, next_token], dim=1)

        # Stop if <EOS> token is predicted
        if next_token.item() == eos_idx:
            break

    # Decode the target tokens back into text
    translated_tokens = tgt.squeeze().tolist()

    # Convert token IDs back to the target language sentence
    translated_sentence = tokenizer.decode(translated_tokens, skip_special_tokens=True)

    return translated_sentence


# Test translation example
for example_idx in range(5):

    example_sentence = test_texts[example_idx]['en']
    refence_sentence = test_texts[example_idx]['fr']
    translated_sentence = translate_sentence(
        model, tokenizer, example_sentence, max_length=max_length, pad_idx=pad_idx, device=device
    )

    print(f"Source sentence: {example_sentence}")
    print(f"Translated sentence: {translated_sentence}")
    print(f"Reference sentence: {refence_sentence}")
    print()

Source sentence: Then we will return with any who may wish to follow us to colonize it definitely, and endow the American Republic with a useful station in this part of the Pacific Ocean!"
Translated sentence: -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- --
Reference sentence: Puis, nous y reviendrons avec tous ceux qui voudront nous suivre pour la coloniser définitivement et doter la république américaine d'une station utile dans cette partie de l'océan Pacifique!

Source sentence: "Hurrah!" exclaimed Pencroft, "and that will be no small present which we shall make to our country!
Translated sentence: -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- --
Reference sentence: -- Hurrah! s'écria Pencroff, et ce

Calculate the metrics of translation quality: [BLEU](https://en.wikipedia.org/wiki/BLEU) score

In [23]:
import torch
from torch.utils.data import DataLoader
from tqdm import tqdm
import sacrebleu

# Function to evaluate the model on the test set
def evaluate_model(model, test_loader, criterion, pad_idx, device):
    model.eval()
    total_loss = 0
    all_references = []
    all_hypotheses = []

    with torch.no_grad():
        for batch in tqdm(test_loader):
            src, tgt = batch['src'].to(device), batch['tgt'].to(device)

            # Prepare input and output sequences
            tgt_input = tgt[:, :-1]  # Input to the decoder
            tgt_output = tgt[:, 1:]  # Target to compare with output

            src_mask = None
            tgt_mask = None

            src_padding_mask = create_padding_mask(src, pad_idx).to(device)
            tgt_padding_mask = create_padding_mask(tgt_input, pad_idx).to(device)
            memory_padding_mask = create_padding_mask(src, pad_idx).to(device)

            # Forward pass
            output = model(src, tgt_input, src_mask, tgt_mask, src_padding_mask, tgt_padding_mask, memory_padding_mask)

            # Compute the loss
            output_flat = output.view(-1, output.size(-1))
            tgt_output_flat = tgt_output.contiguous().view(-1)
            loss = criterion(output_flat, tgt_output_flat)
            total_loss += loss.item()

            # Generate predictions (greedy decoding)
            decoded_sentences = torch.argmax(output, dim=-1)

            # Convert decoded sentences to strings and store for BLEU score calculation
            for i in range(decoded_sentences.size(0)):
                # Remove padding tokens, BOS, and EOS tokens from the predictions
                pred_tokens = decoded_sentences[i].tolist()
                pred_tokens = [token for token in pred_tokens if token != pad_idx]

                # Convert tokens back to string
                hypothesis = tokenizer.decode(pred_tokens, skip_special_tokens=True)

                # Store the generated sentence (hypothesis)
                all_hypotheses.append(hypothesis)

                # Convert target tokens to string
                target_tokens = tgt[i, 1:].tolist()  # Remove <BOS>
                target_tokens = [token for token in target_tokens if token != pad_idx]

                reference = tokenizer.decode(target_tokens, skip_special_tokens=True)

                # Store the reference sentence
                all_references.append([reference])  # sacrebleu expects a list of references

    # Compute BLEU score
    bleu = sacrebleu.corpus_bleu(all_hypotheses, all_references)
    avg_loss = total_loss / len(test_loader)

    print(f"Test Loss: {avg_loss}")
    print(f"BLEU Score: {bleu.score}")

    return avg_loss, bleu.score


# Running the evaluation
evaluate_model(model, test_dataloader, criterion, pad_idx, device)


100%|██████████| 3178/3178 [02:54<00:00, 18.16it/s]


Test Loss: 6.432986984468992
BLEU Score: 0.0


(6.432986984468992, 0.0)